In [1]:
import pandas as pd

In [2]:
matches_df = pd.read_csv('../../data/raw/matches_combined.csv', index_col=0)

In [3]:
matches_df['date'] = pd.to_datetime(matches_df['date'])

In [4]:
matches_df['match_hour'] = matches_df['time'].replace(":.+", "", regex=True).astype('int')

In [5]:
matches_df['day_code'] = matches_df['date'].dt.day_of_week

In [6]:
matches_df['week'] = matches_df.apply(lambda x: x['round'].split(' ')[-1], axis=1)

In [7]:
bundesliga_teams = matches_df[matches_df['comp'] == 'Bundesliga']['team_home'].unique()

In [8]:
# Ensure date is datetime
matches_df['date'] = pd.to_datetime(matches_df['date'])

# Sort full match list chronologically
matches_df = matches_df.sort_values('date').reset_index(drop=True)

# Track last game dates per team
last_game_date = {}

# Only update `days_since_home` and `days_since_away` for Bundesliga matches
matches_df['days_since_home'] = None
matches_df['days_since_away'] = None

for index, row in matches_df.iterrows():
    match_date = row['date']
    home_team = row['team_home']
    away_team = row['team_away']
    comp = row['comp']

    # If it's a Bundesliga game → calculate days since last match
    if comp == 'Bundesliga':
        # Home team
        if home_team in last_game_date:
            matches_df.at[index, 'days_since_home'] = (match_date - last_game_date[home_team]).days
        else:
            matches_df.at[index, 'days_since_home'] = 0

        # Away team
        if away_team in last_game_date:
            matches_df.at[index, 'days_since_away'] = (match_date - last_game_date[away_team]).days
        else:
            matches_df.at[index, 'days_since_away'] = 0

    # Always update last seen match for both teams (regardless of competition)
    last_game_date[home_team] = match_date
    last_game_date[away_team] = match_date


In [9]:
matches_df[matches_df['comp'] == 'Bundesliga'].reset_index(drop=True).to_csv('../../data/raw/matches_buli.csv')
matches_df = matches_df[matches_df['comp'] == 'Bundesliga']

## Goal ratio

In [10]:
matches_df['goals_home'] = matches_df['goals_home'].astype(float)
matches_df['goals_away'] = matches_df['goals_away'].astype(float)
matches_df['goal_ratio_home'] = (matches_df['goals_home'] + 1) / (matches_df['goals_away'] + 1)
matches_df['goal_ratio_away'] = (matches_df['goals_away'] + 1) / (matches_df['goals_home'] + 1)

In [13]:
def rolling_stats(cols, window):
    for team in matches_df['team_away'].unique():
        #get the data frame of one team with all matches (home and away) and also the indexes of the matches
        team_df = matches_df[(matches_df['team_away'] == team) | (matches_df['team_home'] == team)].sort_values('date')
        match_indexes = list(team_df.index)

        #set all values befor our window to nan to delete later
        for early_idx in match_indexes[:window]:
            venue_early = 'home' if matches_df.loc[early_idx]['team_home'] == team else 'away'
            for col in cols:
                matches_df.at[early_idx, f"{col}_{venue_early}_rolling_{window}"] = float('nan')

        #iterate over all matches of a team starting the game after 'window' so we can calc
        # stats from the 'window' last matches
        for match_idx, row in team_df.iloc[window:].iterrows():
            rolling_cols = {f"{col}": 0.0 for col in cols}
            
            #get the indexes of the last matches before this match
            match_list_index = match_indexes.index(match_idx)
            last_matches_indexes = match_indexes[match_list_index - window:match_list_index]

            #sum up the stats from the last matches of this team for all rolling columns
            for last_match_index in last_matches_indexes:
                match = matches_df.loc[last_match_index]
                for col in cols:
                    if match['team_home'] == team:
                        rolling_cols[col] += match[f"{col}_home"] / window
                    else:
                        rolling_cols[col] += match[f"{col}_away"] / window

            #set the averag roling stats to the original dataframe
            venue = 'home' if row['team_home'] == team else 'away'
            for col in cols:
                matches_df.at[match_idx, f"{col}_{venue}_rolling_{window}"] = rolling_cols[col]

rolling_stats(['xg', 'sh', 'sot', 'poss', 'goal_ratio', 'elo'], 3)
rolling_stats(['goal_ratio'], 1)

matches_df['elo_rolling_diff'] = matches_df['elo_home_rolling_3'] - matches_df['elo_away_rolling_3']
matches_df['goal_ratio_diff'] = matches_df['goal_ratio_home_rolling_3'] - matches_df['goal_ratio_away_rolling_3']

matches_df['home_won_game'] = matches_df['result_home'].apply(lambda x: 1 if x == 'W' else 0)

matches_df.dropna().to_csv('../../data/raw/matches_rolling.csv')

